In [ ]:
# Import necessary libraries  
import pandas as pd
import numpy as np

# Load the labels dataset from the .csv file
arrestdf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Arrest.csv')
explosiondf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Explosion.csv')
shootingdf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Shooting.csv')
fightdf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Fight.csv')
normaldf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Normal.csv')
roadaccidentdf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Roadaccidents.csv')
stealingdf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Stealing.csv')
vandalismdf = pd.read_csv('/kaggle/input/dcsass-dataset/DCSASS Dataset/Labels/Vandalism.csv')

In [ ]:
def process_dataframe(df, category_name):
    first_col = [col for col in df.columns if f"{category_name}" in col][0]
    
    df['video_path'] = df[first_col]
    df = df.drop(columns=[first_col, category_name])
    if '0' in df.columns:
        df = df.rename(columns={'0': 'label'})
    
    df['video_path'] = df['video_path'].apply(lambda x: f"{x}.mp4")
    return df

categories = ["arrest", "Explosion", "Fight", "normal", "roadaccidents", 
              "shooting", "Stealing", "vandalism"]

for category in categories:
    df_name = f"{category.lower()}df"
    
    if category.lower() + "df" in globals():
        df_to_process = globals()[category.lower() + "df"]
    else:
        df_to_process = globals()[category.capitalize() + "df"]
    
    globals()[df_name] = process_dataframe(df_to_process, category)    
    print(f"Processed {category} dataframe")


In [ ]:
shootingdf = process_dataframe(shootingdf, "shooting")
arrestdf = process_dataframe(arrestdf, "arrest")
explosiondf = process_dataframe(explosiondf, "explosion")
fightdf = process_dataframe(fightdf, "fight")
roadaccidentdf = process_dataframe(roadaccidentdf, "roadaccident")
stealingdf = process_dataframe(stealingdf, "stealing")
vandalismdf = process_dataframe(vandalismdf, "vandalism")

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from pytorchvideo.models.hub import i3d_r50
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
    ShortSideScale,
    UniformCropVideo,
)

LABELS = ["arrest", "Explosion", "Fight", "normal", "roadaccidents", "shooting", "Stealing", "vandalism"]

class CustomI3D(nn.Module):
    def __init__(self, num_classes=8, dropout_prob=0.5):
        super(CustomI3D, self).__init__()
        self.base_model = i3d_r50(pretrained=True)
        
        in_features = self.base_model.blocks[5].proj.in_features
        self.base_model.blocks[5].proj = nn.Sequential(
            nn.Dropout(dropout_prob),
            nn.Linear(in_features, num_classes)
        )
    
    def forward(self, x):
        return self.base_model(x)

class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, transform=None, clip_duration=2.0):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        self.clip_duration = clip_duration
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        
        try:
            video = EncodedVideo.from_path(video_path)
            
            start_sec = 0.0
            end_sec = start_sec + self.clip_duration
            video_data = video.get_clip(start_sec, end_sec)
            
            if self.transform:
                video_data = self.transform(video_data)
            
            return video_data["video"], label
            
        except Exception as e:
            print(f"Error loading video {video_path}: {e}")
            return torch.zeros(3, 8, 224, 224), label

def get_video_transform(clip_duration=2.0, frames_per_second=30):
    num_frames = int(clip_duration * frames_per_second)
    
    transform = ApplyTransformToKey(
        key="video",
        transform=transforms.Compose([
            UniformTemporalSubsample(num_frames),
            ShortSideScale(size=256),
            transforms.CenterCrop(224),
            transforms.Normalize(
                mean=[0.45, 0.45, 0.45],
                std=[0.225, 0.225, 0.225]
            ),
        ]),
    )
    return transform

def prepare_dataloaders(train_videos, train_labels, val_videos, val_labels, batch_size=8):
    train_transform = get_video_transform()
    val_transform = get_video_transform()
    
    train_dataset = VideoDataset(train_videos, train_labels, transform=train_transform)
    val_dataset = VideoDataset(val_videos, val_labels, transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return train_loader, val_loader

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device):
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_i3d_model.pth')
            print(f'Saved best model with validation accuracy: {best_val_acc:.4f}')
        
        print('-' * 60)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

batch_size = 8
num_epochs = 15
learning_rate = 1e-4
weight_decay = 1e-5


train_loader, val_loader = prepare_dataloaders(
    train_videos, train_labels, val_videos, val_labels, batch_size
)

model = CustomI3D(num_classes=len(LABELS), dropout_prob=0.5)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device)

print("Training completed!")

# save best model
model.load_state_dict(torch.load('best_i3d_model.pth'))
model.eval()

def predict_video(video_path):
    transform = get_video_transform()
    video = EncodedVideo.from_path(video_path)
    video_data = video.get_clip(0.0, 2.0)
    video_data = transform(video_data)
    
    with torch.no_grad():
        inputs = video_data["video"].unsqueeze(0).to(device)
        outputs = model(inputs)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(outputs, dim=1).item()
        
    print(f"Predicted class: {LABELS[predicted_class]}")
    print(f"Probabilities: {probabilities.cpu().numpy()}")
    
    return LABELS[predicted_class], probabilities.cpu().numpy()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [27]:
model.save("my_model_final.h5")

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
